In [1]:
## https://hydrofunctions.readthedocs.io/en/master/introduction.html
import hydrofunctions as hf
import pandas as pd

In [2]:
## https://help.waterdata.usgs.gov/code/parameter_cd_query?fmt=rdb&inline=true&group_cd=%
## 00400	Physical	pH, water, unfiltered, field, standard units	Agree					Total		 	pH	std units
## 63680	Physical	Turbidity, water, unfiltered, monochrome near infra-red LED light, 780-900 nm, detection angle 90 +-2.5 degrees, formazin nephelometric units (FNU)	USGS use and no use by EPA					Total		 	Turbidity	FNU
## 00010	Physical	Temperature, water, degrees Celsius	Agree							 	Temperature, water	deg C

In [3]:
## dates
start_date = "2000-01-01"
end_date = "2022-12-31"

## water data parameters (pH, turbidity, temperature)
parameters = ["00400","63680","00010"]

In [4]:
def get_water_quality_data(state,start_date,end_date, parameters):
  state = hf.NWIS(stateCd = state, start_date=start_date, end_date=end_date, parameterCd=parameters)
  state_df = state.df()
  state_df = state_df.reset_index()

  # get turbidity values
  turbidity_columns = [col for col in state_df.columns if ':63680' in col and not col.endswith('_qualifiers')]
  turbidity = state_df[turbidity_columns].mean(axis=1)

  # get temperature values
  temperature_columns = [col for col in state_df.columns if ':00010' in col and not col.endswith('_qualifiers')]
  temperature = state_df[temperature_columns].mean(axis=1)

  # get pH values
  ph_columns = [col for col in state_df.columns if ':00400' in col and not col.endswith('_qualifiers')]
  ph = state_df[ph_columns].mean(axis=1)

  # water quality for state
  state_water_quality = pd.DataFrame({'datetimeUTC': state_df['datetimeUTC'], 
                          'turbidity value': turbidity,
                          'temperature value': temperature,
                          'pH value': ph})
  
  state_water_quality['datetimeUTC'] = pd.to_datetime(state_water_quality['datetimeUTC'])
  state_water_quality = state_water_quality.set_index('datetimeUTC')
  state_water_quality_monthly = state_water_quality.resample('M').mean()
  return state_water_quality_monthly


In [5]:
## California

california_water_quality_monthly = get_water_quality_data('CA',start_date,end_date, parameters)
california_water_quality_monthly

Requested data from https://waterservices.usgs.gov/nwis/dv/?format=json%2C1.1&stateCd=CA&parameterCd=00400%2C63680%2C00010&startDT=2000-01-01&endDT=2022-12-31


ERROR:hydrofunctions.exceptions:It is not possible to determine the frequency for one of the datasets in this request. This dataset will be set to a frequency of 0 minutes
/usr/local/lib/python3.9/dist-packages/hydrofunctions/hydrofunctions.py:103: HydroUserWarning: It is not possible to determine the frequency for one of the datasets in this request. This dataset will be set to a frequency of 0 minutes
  warnings.warn(
ERROR:hydrofunctions.exceptions:It is not possible to determine the frequency for one of the datasets in this request. This dataset will be set to a frequency of 0 minutes
/usr/local/lib/python3.9/dist-packages/hydrofunctions/hydrofunctions.py:103: HydroUserWarning: It is not possible to determine the frequency for one of the datasets in this request. This dataset will be set to a frequency of 0 minutes
  warnings.warn(
ERROR:hydrofunctions.exceptions:One or more datasets in this request is going to be 'upsampled' to 1 days 00:00:00 because the data were collected at a 

,turbidity value,temperature value,pH value
datetimeUTC,,,
2000-01-31 00:00:00+00:00,NaN,9.178916,8.317204
2000-02-29 00:00:00+00:00,NaN,9.914265,8.116379
2000-03-31 00:00:00+00:00,NaN,11.156489,8.110941
2000-04-30 00:00:00+00:00,NaN,13.570469,7.831667
2000-05-31 00:00:00+00:00,NaN,15.317995,7.815430
...,...,...,...
2022-08-31 00:00:00+00:00,3.378961,19.537764,7.898180
2022-09-30 00:00:00+00:00,3.206449,18.180644,7.854580
2022-10-31 00:00:00+00:00,2.251239,15.170036,7.837428
